In [1]:
import pandas as pd
import os
wdir = '/Users/candicewithrow/code/niffler/niffler'


In [2]:
pwd

'/Users/candicewithrow/code/niffler/niffler'

In [3]:
import json
with open(os.path.join(wdir,'output/full_02_03_2019.json'), 'r') as f:
                nested_data = json.load(f)
           

In [4]:
for rec in nested_data:
    rec['post_id'] = rec['id']
    rec.pop('id', None)
    for c in rec['comments']:
        c['cid'] = c['id']
        c.pop('id', None)
        if 'replies' in c.keys():
            for r in c['replies']:
                if 'id' in r.keys():
                    r['rid'] = r['id']
                    r.pop('id', None)
    

In [5]:
nested_data

[{'comments': [{'cid': '10155966481665857',
    'created_time': '2019-01-24T20:05:45+0000',
    'message': 'Not ocular but yes migraines are my primary reaction to sulphites. Best wishes',
    'replies': [{'created_time': '2019-01-25T10:57:54+0000',
      'message': 'Clare Ross same',
      'rid': '10155967531485857'}]},
   {'cid': '10155966792785857',
    'created_time': '2019-01-24T22:58:51+0000',
    'message': 'I  have  ocular  migraines  occasionally,  but  never  thought  to  link  them  to  sulfites.  I  do  remember  once  when  it  happened  i I was  getting  over  a  cold.  I  do  get  headaches  that  can  progress  to  migraines  from  certain  sulfites.'},
   {'cid': '10155966795795857',
    'created_time': '2019-01-24T23:02:44+0000',
    'message': 'My ocular migraines decreased significantly after cutting out sulfites, and migraines are a sure thing if I happen to eat them. I have meds and migraine contacts to cope'},
   {'cid': '10155966868725857',
    'created_time': '

In [6]:
posts=pd.read_json(os.path.join(wdir,'output/full_02_03_2019.json'), orient='records')

In [12]:
from pandas.io.json import json_normalize
posts = json_normalize(nested_data)[['post_id','message','updated_time']]
comments = json_normalize(nested_data, record_path='comments', meta=['post_id'])
comments.head(3)

,cid,created_time,message,replies,post_id
0,10155966481665857,2019-01-24T20:05:45+0000,Not ocular but yes migraines are my primary re...,"[{'message': 'Clare Ross same', 'created_time'...",64565870856_10155966469605857
1,10155966792785857,2019-01-24T22:58:51+0000,"I have ocular migraines occasionally, but...",NaN,64565870856_10155966469605857
2,10155966795795857,2019-01-24T23:02:44+0000,My ocular migraines decreased significantly af...,NaN,64565870856_10155966469605857


In [8]:
full_df = pd.merge(posts, comments, on='post_id')
full_df[['message_x','message_y', 'created_time', 'updated_time']].head(5)

,message_x,message_y,created_time,updated_time
0,Do any of you get ocular migraines? Any relati...,Not ocular but yes migraines are my primary re...,2019-01-24T20:05:45+0000,2019-02-02T19:04:06+0000
1,Do any of you get ocular migraines? Any relati...,"I have ocular migraines occasionally, but...",2019-01-24T22:58:51+0000,2019-02-02T19:04:06+0000
2,Do any of you get ocular migraines? Any relati...,My ocular migraines decreased significantly af...,2019-01-24T23:02:44+0000,2019-02-02T19:04:06+0000
3,Do any of you get ocular migraines? Any relati...,My daughter’s primary reaction to sulfites is ...,2019-01-24T23:56:29+0000,2019-02-02T19:04:06+0000
4,Do any of you get ocular migraines? Any relati...,I used to get ocular migraines but haven’t had...,2019-01-25T00:44:44+0000,2019-02-02T19:04:06+0000


In [13]:
full_df['post_updated']=pd.to_datetime(full_df['updated_time'])
full_df['message_created']=pd.to_datetime(full_df['created_time'])
full_df[['message_x','message_y', 'post_updated', 'message_created']].head(2)

,message_x,message_y,post_updated,message_created
0,Do any of you get ocular migraines? Any relati...,Not ocular but yes migraines are my primary re...,2019-02-02 19:04:06,2019-01-24 20:05:45
1,Do any of you get ocular migraines? Any relati...,"I have ocular migraines occasionally, but...",2019-02-02 19:04:06,2019-01-24 22:58:51


In [10]:
full_df['updated_time'].apply(lambda x:pd.to_datetime(x).date().strftime('%m-%d-%Y')).head()


0    02-02-2019
1    02-02-2019
2    02-02-2019
3    02-02-2019
4    02-02-2019
Name: updated_time, dtype: object